<a href="https://colab.research.google.com/github/Lolit-78/T3-PROYECT/blob/main/chatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de Hugging Face y otras librerías
!pip install --upgrade transformers datasets scikit-learn -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

# Cargar el dataset público de intenciones bancarias
dataset = load_dataset("banking77")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/298k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/93.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

# Usamos tokenizer base de BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Función para tokenizar cada texto
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

# Aplicar la tokenización
encoded_dataset = dataset.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

Map:   0%|          | 0/3080 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification

# Número de clases (77 intenciones bancarias)
num_labels = 77

# Modelo BERT adaptado para clasificación
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Función para cálculo de métricas
def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }


In [6]:
from transformers import TrainingArguments

# Argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir="./banking77_model",
    do_train=True,
    do_eval=True,
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch"  # guarda al final de cada época
)


In [7]:
pip install wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.7/357.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.3 MB/s eta 0:00:00


In [8]:

!pip install --upgrade transformers


In [12]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import load_dataset

# 1. Cargar modelo y tokenizer
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# 2. Cargar y tokenizar datos
dataset = load_dataset("imdb")
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 3. Usar DataCollator (evita pasar tokenizer directamente al Trainer)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 4. Configurar TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none"
)

# 5. Crear Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(2000)),  # Ejemplo reducido
    eval_dataset=tokenized_datasets["test"].select(range(500)),
    data_collator=data_collator,
    tokenizer=tokenizer
)

# 6. Entrenar modelo
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

/tmp/ipython-input-1080782385.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.840423
2,No log,0.557776
3,No log,0.348082


TrainOutput(global_step=375, training_loss=0.22716796875, metrics={'train_runtime': 422.8686, 'train_samples_per_second': 14.189, 'train_steps_per_second': 0.887, 'total_flos': 783230052978432.0, 'train_loss': 0.22716796875, 'epoch': 3.0})

In [17]:
# Guardar modelo y tokenizer en carpeta local
model.to('cpu').save_pretrained("modelo_banking77")
tokenizer.save_pretrained("modelo_banking77")

('modelo_banking77/tokenizer_config.json',
 'modelo_banking77/special_tokens_map.json',
 'modelo_banking77/vocab.txt',
 'modelo_banking77/added_tokens.json',
 'modelo_banking77/tokenizer.json')

In [18]:
from transformers import pipeline

# Crear pipeline de clasificación con el modelo entrenado
clasifier = pipeline("text-classification", model="modelo_banking77", tokenizer="modelo_banking77")

# Frase de prueba
phrase = "I want to check my account balance"
result = clasifier(phrase)
print("Prediction:", result)



Device set to use cpu


Prediction: [{'label': 'LABEL_0', 'score': 0.8863343596458435}]


In [19]:
phrases = [
    "I lost my debit card and need a replacement",
    "I want to transfer 500 dollars to another account",
    "How can I check my account balance?",
    "Where is the closest ATM near me?",
    "I forgot my PIN code",
    "Can I open a savings account online?",
    "What is the interest rate for home loans?",
    "I need help logging into my online banking",
    "Cancel my credit card immediately"
]

for phrase in phrases:
    result = clasifier(phrase)
    print(f"\nCustomer: {phrase}")
    print("Detected intent:", result)



Customer: I lost my debit card and need a replacement
Detected intent: [{'label': 'LABEL_0', 'score': 0.9946268200874329}]

Customer: I want to transfer 500 dollars to another account
Detected intent: [{'label': 'LABEL_0', 'score': 0.8724785447120667}]

Customer: How can I check my account balance?
Detected intent: [{'label': 'LABEL_0', 'score': 0.9850926995277405}]

Customer: Where is the closest ATM near me?
Detected intent: [{'label': 'LABEL_0', 'score': 0.8918213248252869}]

Customer: I forgot my PIN code
Detected intent: [{'label': 'LABEL_0', 'score': 0.9939344525337219}]

Customer: Can I open a savings account online?
Detected intent: [{'label': 'LABEL_0', 'score': 0.7156692147254944}]

Customer: What is the interest rate for home loans?
Detected intent: [{'label': 'LABEL_0', 'score': 0.9779687523841858}]

Customer: I need help logging into my online banking
Detected intent: [{'label': 'LABEL_1', 'score': 0.5563928484916687}]

Customer: Cancel my credit card immediately
Detected